In [13]:
# pip install google-genai google-cloud-aiplatform

import vertexai
from vertexai import rag
from vertexai.generative_models import GenerativeModel, Tool, GenerationConfig

In [21]:
# Replace using your data
PROJECT_ID = "trim-tributary-*****-***"
LOCATION = "europe-west2"
RAG_CORPUS_NAME = "projects/trim-tributary-*****-***/locations/europe-west2/ragCorpora/****"

RAG_FILE_IDS = [] # file ids to search in. if [] the model will look through all files

In [25]:
# setup API
vertexai.init(project=PROJECT_ID, location=LOCATION)
retrieval_config = rag.RagRetrievalConfig(top_k=5) # tuning param: number of most relevant parts of the text

rag_store = rag.VertexRagStore(
        rag_resources=[
            rag.RagResource(
                rag_corpus=RAG_CORPUS_NAME,
                rag_file_ids=RAG_FILE_IDS,
            )
        ],
        rag_retrieval_config=retrieval_config,
    )

rag_tool = Tool.from_retrieval(
        retrieval=rag.Retrieval(source=rag_store)
    )

model = GenerativeModel(
        model_name="gemini-2.5-flash",
        tools=[rag_tool]
    )

In [40]:
#question = "Give information about Package Contents"
question = "Make a  summary of the contents."
prompt = (
        "Answer ONLY based on the documents from the connected RAG corpus. "
        "If the required information is not present in the documents, write you don't have information.\n\n"
        f"Question: {question}"
    )


In [41]:
response = model.generate_content(
        prompt,
        generation_config=GenerationConfig(
            temperature=0.0,  # param to turn off context outside files inside storage
        ),
    )

print(response.candidates[0].content.parts[0].text)

This document is an electric kettle user manual. It provides an introduction to the kettle's purpose, lists package contents (kettle, power base, manual), and outlines safety instructions such as not immersing the kettle in liquid, not turning it on without water, keeping it away from children and pets, and using a functioning power outlet.

The manual details how to use the kettle: place it on a flat surface, fill it with water within MIN/MAX markings, put it back on the base, switch it on, and wait for automatic shut-off. It also covers cleaning and maintenance, advising regular descaling (every 1-2 months) with citric acid or a descaling agent, and not using abrasive materials.

Technical specifications include power (1800–2200 W), capacity (1.5–2 L), and body material (glass/plastic/stainless steel). Common issues and solutions are addressed, such as checking the outlet if it won't turn on, descaling if it boils slowly, and rinsing if there's a strange taste. Finally, it states a s

In [ ]:
# Manual content
'''
Electric Kettle User Manual

1. Introduction

This simple guide will help you quickly understand how to use your
electric kettle safely and efficiently. The kettle is designed for fast
water boiling in everyday household use.

2. Package Contents

-   Electric kettle
-   Power base
-   Quick start manual (you’re reading the extended version 🙂)

3. Safety Instructions

-   Do not immerse the kettle in water or any other liquid.
-   Do not turn it on without water inside.
-   Keep it away from children and pets.
-   Always use a functioning power outlet.

4. How to Use

1.  Place the kettle on a flat surface.
2.  Remove it from the base and fill it with water within the MIN/MAX
    markings.
3.  Put the kettle back on the base.
4.  Switch it on by pressing the power lever.
5.  Wait for the automatic shut‑off after boiling.

5. Cleaning and Maintenance

-   Descale the kettle regularly (every 1–2 months).
-   Use citric acid solution or a descaling agent.
-   Do not use abrasive cleaning materials.

6. Technical Specifications

-   Power: 1800–2200 W
-   Capacity: 1.5–2 L
-   Body material: glass/plastic/stainless steel (depending on model)

7. Common Issues and Solutions

-   Won’t turn on: check the outlet and ensure the kettle is seated
    properly on the base.
-   Boils slowly: descaling may be needed.
-   Strange taste: rinse the kettle several times with clean water.

8. Warranty

Standard warranty — 1 year from the date of purchase. Keep your receipt
for service.

Thank you for using our kettle! May your water boil fast and your mood
stay warm ☕
'''